### Objective:
- Read csv file from partitioned data

In [1]:
import pandas as pd
import numpy as np

In [2]:
## read csv file
# df_a = pd.read_csv("Project_AWS_Web_Crawl/data_sources/columns.csv/part-00046-3c5edd89-b8f2-4f96-b3bb-c044c9c50f9c-c000.csv")

# Read the partitioned Parquet file
#df_x= pd.read_parquet("Project_AWS_Web_Crawl/data_sources/first_AWS_web_Crawl_partitioned/part-00000-e767fe13-c5f2-4f7f-b7f4-4730bb02c5d3-c000.snappy.parquet")

In [3]:
# halve= df_x[: len(df_x)//2]
#df_x[ : 1000].to_csv("sample_webCrawl.csv" , index= False)
#df_x.to_csv("full_sample_webCrawl.csv" , index= False)

### NOTE:
- Ignore the first column header, header= 1, this starts with second row
- Adjusting with new columns names
- We will further split the `URL` into four sub columns

In [159]:
# df_x= pd.read_csv("sample_webCrawl.csv",
#                  names= ["url_surtkey", "url", "url_host_tld", "url_host_2nd_last_part", "url_host_3rd_last_part", ""])

## asign the header
new_col_names= ["c1_", "c2_", "c3_", "url", "mime", "mime_type", "status", "digest_offset", "length", "offset", "filename","content_charset", "language"]

# sample of 10000 rows
#df_x= pd.read_csv("sample_webCrawl.csv", names= new_col_names, header=1)

# full file 
df_x= pd.read_csv("full_sample_webCrawl.csv", names= new_col_names, header=1)

In [169]:
df_x.head(5)

,c1_,c2_,c3_,url,mime,mime_type,status,digest_offset,length,offset,filename,content_charset,language
0,247,175,1,3)/post/naga321-agen-mega-slot-99 202312100823...,"""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""200""","""digest"": ""WWYHNELFKRNADYS5VKQAO5LS2FNQNZMX""","""length"": ""18571""","""offset"": ""98699237""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""charset"": ""UTF-8""","""languages"": ""ind"
1,22,231,111,"47:8083)/cate/1077.html 20231207062957 {""url"":...","""mime"": ""unk""","""mime-detected"": ""text/html""","""status"": ""302""","""digest"": ""3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ""","""length"": ""562""","""offset"": ""8684""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""redirect"": ""http://47.111.231.22:8083/0-1077...",NaN
2,229,132,118,45)/desktop/arcade/microgaming 20231210173131 ...,"""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""200""","""digest"": ""ZTI5WXHBYA6PO4RMH6HNZN3V7NLYLUSH""","""length"": ""31162""","""offset"": ""98134489""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""charset"": ""UTF-8""","""languages"": ""ind""}"
3,163,30,85,"1)/signin?theme=project 20231128164124 {""url"":...","""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""200""","""digest"": ""4GRWO5GFA24SSYJ7SKCAN5CPIDGPMUDC""","""length"": ""2238""","""offset"": ""51154""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""charset"": ""UTF-8""","""languages"": ""zho""}"
4,13,158,44,"64)/robots.txt 20231129091744 {""url"": ""https:/...","""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""404""","""digest"": ""HQFPHHWLG5J4L7XDWU6QMPDSQYAZ5LB3""","""length"": ""539""","""offset"": ""790668""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...",NaN,NaN


In [161]:
len(df_x)

627736

In [162]:
df_x["url"][9]

'61)/updoc/prakad640708.pdf 20231211081242 {"url": "http://61.19.73.34/updoc/prakad640708.pdf"'

In [163]:
df_x["url"][3]

'1)/signin?theme=project 20231128164124 {"url": "http://1.85.30.163/signin?theme=project"'

### substring from column `url`
- Breaking this column name to four sub columns:
    - IP_Part_Port
    - Resource_Path
    - Timestamp
    - Full_URL

- Example: splitting this name `47:8083)/cate/1077.html 20231207062957 {"url": "http://47.111.231.22:8083/cate/1077.html`
    - IP_Part_Port: 47:8083
    - Resource_Path: /cate/1077.html
    - TimeStamp: 20231207062957
    - Full_Url: "url": "http://47.111.231.22:8083/cate/1077.html

#### Create new Sub columns

In [23]:
test_url= df_x["url"][3]

In [24]:
full_url

['185)/year/2021',
 '20231211150323',
 '{"url":',
 '"http://185.215.164.57/year/2021/"']

In [31]:
url_= df_x["url"][50]
full_url= url_.split(' ')

ip_resource= full_url[0].split(")")

# get IP path
ip_path = ip_resource[0]
# Get Resource path
resource_path = ip_resource[1]

# get timestamp
time_stamp= full_url[1]
# Get the full URL
url = full_url[3].replace('"', '')


In [36]:
full_url
print('\n')
resource_path

'/year/2021'

In [300]:
## funciton to perform substring on Full URL path
def process_url(df):

    ## replace the NaN with NONE value
    # df= df.dropna()

    #df["url"]= df["url"].replace({ np.nan: 0})

    ## list of substrings
    ip_path= []
    resource_path= []
    time_stamp = []
    url= []

    
    # iterate over each row
    for j in range(0, len(df)):

#        if pd.notna(df["url"][j]):

        # split the url
        full_url= df["url"][j].split(' ')        
            
        # Get full path
        ip_resource= full_url[0].split(")")
        
        # get IP path
        ip_path.append(ip_resource[0])
        # Get Resource path
        resource_path.append(ip_resource[1])
    
        # get timestamp
        time_stamp.append(full_url[1])
        # Get the full URL
        url.append(full_url[3].replace('"', ''))

#        else:
#            continue
            
    df_new= df.copy()

    # Append the values
    df_new["Ip_path"]= ip_path
    df_new["Resource_path"]= resource_path
    df_new["TimeStamp"]= time_stamp
    df_new["URL"] = url

    ## drop URL column
    df_new= df_new.drop(columns= ["url"])

    return df_new

In [288]:
df_half= df_x[998:]

In [289]:
df_half["url"].head(4)

998     54)/terminos-de-uso 20231130113127 {"url": "ht...
999     119)/searchdemo/searchvdodemo7.php 20231208014...
1000    202:888)/?id=9827 20231206231934 {"url": "http...
1001    111)/director/sagar-chandra 20231208181455 {"u...
Name: url, dtype: object

### Check for `NaN` values in Column URL

In [290]:
# df_none= df_half[df_half["url"].isna()]
# df_none["url"]= df_none["url"].replace({ np.nan: 0})

In [291]:
%%time
process_url(df_half)

KeyError: 0

In [297]:
df_copy = df_x.copy()

df_copy= df_copy[pd.notna(df_copy["url"])]

In [299]:
len(df_x),len(df_copy)

(627736, 627728)

In [301]:
%%time
process_url(df_copy)

IndexError: list index out of range

In [318]:
df_copy.tail(3)

,c1_,c2_,c3_,url,mime,mime_type,status,digest_offset,length,offset,filename,content_charset,language
627733,app,web,faxlibmtuj)/film-la-mano-de-dios-streaming-ita...,"""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""200""","""digest"": ""CXDJVTSIGMANJ7RTANUUAUNUVBA7HKDB""","""length"": ""9088""","""offset"": ""296662863""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""charset"": ""UTF-8""","""languages"": ""ita","eng""}"
627734,app,web,faxlibiwyzh)/aparato-reproductor-primaria-pdf-...,"""mime"": ""warc/revisit""","""status"": ""304""","""length"": ""703""","""offset"": ""10342703""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...",NaN,NaN,NaN,NaN,NaN
627735,app,web,faxlibkptg)/korff31546j/992084.html 2023121007...,"""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""200""","""digest"": ""BNODFYP4PYGKWJJMKE5K5CGPM7E2WEVN""","""length"": ""6675""","""offset"": ""273481327""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""charset"": ""UTF-8""","""languages"": ""spa","eng""}"


In [320]:
df_copy.head(3)

,c1_,c2_,c3_,url,mime,mime_type,status,digest_offset,length,offset,filename,content_charset,language
0,247,175,1,3)/post/naga321-agen-mega-slot-99 202312100823...,"""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""200""","""digest"": ""WWYHNELFKRNADYS5VKQAO5LS2FNQNZMX""","""length"": ""18571""","""offset"": ""98699237""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""charset"": ""UTF-8""","""languages"": ""ind"
1,22,231,111,"47:8083)/cate/1077.html 20231207062957 {""url"":...","""mime"": ""unk""","""mime-detected"": ""text/html""","""status"": ""302""","""digest"": ""3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ""","""length"": ""562""","""offset"": ""8684""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""redirect"": ""http://47.111.231.22:8083/0-1077...",NaN
2,229,132,118,45)/desktop/arcade/microgaming 20231210173131 ...,"""mime"": ""text/html""","""mime-detected"": ""text/html""","""status"": ""200""","""digest"": ""ZTI5WXHBYA6PO4RMH6HNZN3V7NLYLUSH""","""length"": ""31162""","""offset"": ""98134489""","""filename"": ""crawl-data/CC-MAIN-2023-50/segme...","""charset"": ""UTF-8""","""languages"": ""ind""}"


### Include the column names to numeric else set to NULL and drop

In [ ]:
df_cp

In [321]:
len(df_copy["c2_"].unique())

383075